## Import

This code is modified from a tutorial made by Stacey Shield: https://github.com/UCTMechatronics/pyomo_tutorials

Here we import all the required libraries to symbolically generate the EOM as well as print stuff to the screen
We also reset the console, getting rid of previous data

In [1]:
# %reset
# import libraries
import sympy as sym
import numpy as np
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

#sym.init_printing()
from IPython.display import display

In [2]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
m = ConcreteModel() # create model using pyomo
N = 35 # number of nodes (much like samples used between a certain time period)
hm=0.5/N # I think this is the time step
m.N = RangeSet(N) 
m.damping=20

mass_b=2 # Mass of the base
mass_l=1 # Mass of the links
len_b=0.3 # Length of the base
m.l_min=0.2 # Links min length
m.l_max=len_b # Links max length
m.F_max=160 # Maximum force that can be applied
m.t_max=12 # Maximum torque
m.w_max=44 # Max angular velocity (omega)
m.dl_max=3
m.distance=0.5

print('Done')

Done


## EoM generation
Here we create our symbolic variables for the EoM of the system.

The symbolic variables include the mass, length and inertia of each of the robot links
the generalized coordinates include: X, Z, th_body, th_leg and the leg length (a prismatic leg). Relative angles are used
The generalized coordinates also include the velocities and accelerations. 

q=Generalized coordinates
dq=velocity of the generalized coordinates
ddq=acceleration of the generalized coordinates

Forces are also defined. This robot has a hip torque, tau, and a prismatic force to lengthen and shorten the leg, F
F is the total force, it contains the actuator force as well as a rebound force (hard-stop)

There is also a vertical and horizontal GRF

In [3]:
# create symbolic variables
g = sym.symbols('g')
mb,ml = sym.symbols(['m_{body}','m_{leg}']) # mass
lb = sym.symbols(['l_{body}']) # length
Inb,Inl = sym.symbols(['I_{body}','I_{leg}']) # moment of intertia

# generalized coordinates
x,z,thb,thl,r = sym.symbols(['x','z','\\theta_{body}','\\theta_{leg}','r']) 
dx,dz,dthb,dthl,dr = sym.symbols(['\dot{x}','\dot{z}','\dot{\\theta}_{body}','\dot{\\theta}_{leg}','\dot{r}']) 
ddx,ddz,ddthb,ddthl,ddr = sym.symbols(['\ddot{x}','\ddot{z}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg}','\ddot{r}']) 

q = sym.Matrix([[x],[z],[thb],[thl],[r]])
dq = sym.Matrix([[dx],[dz],[dthb],[dthl],[dr]])
ddq = sym.Matrix([[ddx],[ddz],[ddthb],[ddthl],[ddr]])

# forces
F,tau,GRFx,GRFz = sym.symbols(['F','\\tau','G_x','G_z']) 
print('Done')

Done


The position of the COM of each link. [x,z] is the COM of the body. Assuming the CoM is half way, with uniform inertia
rb is the CoM of the body
rl is the CoM of the leg

the body is horizontal, so the leg attaches to the CoM of the body

In [4]:
# STEP 1: position vector (COM of each link), as well as other required positions. 
rb = sym.Matrix([[x],
                [z]])

rl = sym.Matrix([[x + (0.5*r)*sym.cos(thl)],
                 [z - (0.5*r)*sym.sin(thl)]])

footx = sym.Matrix([x + r*sym.cos(thl)])
footz = sym.Matrix([z - r*sym.sin(thl)]) 
footz = footz[0].simplify()
foot_pos= sym.Matrix([[footx],[0],[footz]])
print('Done')

Done


This leads to the following COM's ...
$$rb = \begin{pmatrix} x \\ z \end{pmatrix}$$
$$rl = \begin{pmatrix} x+(0.5*r)cos(\theta_l) \\ z-(0.5*r)sin(\theta_l) \end{pmatrix}$$
$$footx = \begin{pmatrix} x+r*cos(\theta_l) \end{pmatrix}$$
$$footz = \begin{pmatrix} z-r*sym.sin(\theta_l) \end{pmatrix}$$
$$foot_{pos} = \begin{pmatrix} footx \\ 0 \\ footz \end{pmatrix}$$

From the positions defined above, calculate the velocities

In [5]:
# STEP 2: generate expressions for the system space velocities from the jacobians
# the Jacobians
Jb = rb.jacobian(q)
Jl = rl.jacobian(q)

vb = Jb*dq
vl = Jl*dq

footdx = footx.jacobian(q)*dq #This is symbolic again, so it doesnt change with the collocation
footdx = footdx[0].simplify()
print('Done')

Done


Using the positions and velocities, calculate the kinetic and potential energy. This is used to generate the EoM using 
Lagrange Dynamics

In [6]:
# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb]])
Ml = sym.Matrix([[ml,ml]])

Ib=sym.Matrix([[Inb]])
Il=sym.Matrix([[Inl]])

Wb=sym.Matrix([[dthb]])
Wl=sym.Matrix([[dthl]])

# linear kinetic energy:
T1 = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml*sym.matrix_multiply_elementwise(vl,vl)
# angular kinetic energy:
T2 =  0.5*Ib*sym.matrix_multiply_elementwise(Wb,Wb) + 0.5*Il*sym.matrix_multiply_elementwise(Wl,Wl) #this should be a 3*3 matrix of MOI and a vector of angular vel... w'*MOI*w

T = sym.Matrix([T1[0]])+sym.Matrix([T2[0]])
# potential energy
V = mb*g*rb[1] + ml*g*rl[1]
print('Done')

Done


The Matrix Mb was created to simplify the code in the following way:

0.5* Mb* sym.matrix_multiply_elementwise(vb,vb) 
$$ = \begin{pmatrix} 0.5 \dot{x}^2m_{body} + 0.5 \dot{z}^2m_{body} \end{pmatrix}$$

Using the normal *mb* will lead to...

0.5* mb* sym.matrix_multiply_elementwise(vb,vb)
$$ = \begin{pmatrix} 0.5 \dot{x}^2m_{body} \\ 0.5 \dot{z}^2m_{body} \end{pmatrix}$$
The x and z element needs to be added to it is just one step shorter

Calculate T (kinetic energy) using...
$$T = 0.5m \times \dot{x}^2$$
do this for linear and angular kinetic energy

Also calculate V (potential energy)...
$$V = mg \times height_{body}$$

In [7]:
print('T =')
display(T)
display('******************************************************************************************************')
print("V =")
display(V)

T =


Matrix([[0.5*I_{body}*\dot{\theta}_{body}**2 + 0.5*I_{leg}*\dot{\theta}_{leg}**2 + 0.5*\dot{x}**2*m_{body} + 0.5*\dot{z}**2*m_{body} + 0.5*m_{leg}*(-0.5*\dot{\theta}_{leg}*r*sin(\theta_{leg}) + 0.5*\dot{r}*cos(\theta_{leg}) + \dot{x})**2 + 0.5*m_{leg}*(-0.5*\dot{\theta}_{leg}*r*cos(\theta_{leg}) - 0.5*\dot{r}*sin(\theta_{leg}) + \dot{z})**2]])

'******************************************************************************************************'

V =


g*m_{body}*z + g*m_{leg}*(-0.5*r*sin(\theta_{leg}) + z)

*********************************************************************************************************************************************************************************************

Use the manipulator equation to calculate the EoM

EOM=M * ddq+C * dq+ G-Q-J.' * GRF;

In [8]:
M=T.jacobian(dq)
M=M.transpose()
M=M.jacobian(dq)              

C  = sym.zeros(len(q),len(q))                                                         
for i in range(len(q)):                                             
    for j in range(len(q)):
        for n in range(len(q)):
            C[i,j] = C[i,j]+ 0.5*(sym.diff(M[i,j],q[n]) + sym.diff(M[i,n],q[j]) - sym.diff(M[j,n],q[i]))*dq[n];

G  = sym.zeros(len(q),1)                                         
for i in range(len(q)):
    G[i] = sym.diff(V,q[i]);       

J=foot_pos.jacobian(q)
GRF_l = J.transpose()*sym.Matrix([[GRFx],[0],[GRFz]])   

Q = sym.zeros(len(q),1)
Q[0]=0                #X
Q[1]=0                #Z
Q[2]=-tau             #thb
Q[3]=tau              #thl
Q[4]=F-dr*m.damping   #r
           
EOM=M*ddq+C*dq+G-Q-GRF_l

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()
    
print('Done')

Done


This is some weird maths that breaks my brain #theBigSad

The EOM needs to be found, it looks as follow
$$EOM=M(q)\ddot{q}+C(q,\dot{q})\dot{q}+G(q)-Q-GRF$$
Lets try to explain some things and see if it helps.

$M$ is the mass matrix. According the most trusted site (Wikipedia), the mass matrix is a symmetric matrix M that expresses the connection between the time derivative $\dot {q}$ of the generalized coordinate vector $q$ of a system and the kinetic energy $T$ of that system. It look something like this:
$$T = \frac{1}{2} \dot{q}^TM\dot{q}$$
So using the power of maths, it can be rearrangend to find $M$

$C$ is the Coriolis matrix. When it needs to be calculated then just copy the provided code ;)


*******************************************

## Lambdify
Here we need to lamdify the instructions. Basically linking symbolic lists to variables as well as function mapping..... It converts the symbolic representation to equations that the optimizer can use.....

In [9]:
# Lambdify


func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml,lb,Inb,Inl,
            x,z,thb,thl,r,
            dx,dz,dthb,dthl,dr,
            ddx,ddz,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFz]
            
lambEOM_x = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_y = sym.lambdify(sym_list,EOMs[1],modules = [func_map]) # Should be the z-direction but fixing this will take effort
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thl = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_r = sym.lambdify(sym_list,EOMs[4],modules = [func_map])

lamb_footz = sym.lambdify(sym_list,footz,modules = [func_map])
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])
print('Done')

Done


All the variables above are not linked to the model and was only used to get the EOM.

The variables for the model will now be in the code down below.

********
*********


## Trajectory optimization
Here we define all the variables for the trajectory optimization problem (do not confuse them with the variables we defined above for the EoM). All variables defined are linked to the model (m). 

N= node, how many time points the trajectory is split into
cN= collocation points, number of sub points per collocation point

q= generalized coordinates for the node points and collocation points
q0= generalized coordinates for the node points only

The time bounds are similarily split up as well

![Drag Racing](collocation.jpg)


In [10]:
# Variables
cN = 3
m.cN =RangeSet(cN)

links = [('body',1),('leg',1)]
m.L = Set(dimen=2, initialize=links)

DOFs = ['x','z','theta_b','theta_l','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

# system coordinates
m.q = Var(m.N, m.cN, m.DOF) # position
m.dq = Var(m.N, m.cN, m.DOF) # velocity
m.ddq = Var(m.N, m.cN, m.DOF) # acceleration

m.q0 = Var(m.N, m.DOF) # position
m.dq0 = Var(m.N, m.DOF) # velocity

m.tt = Var(m.N, m.cN)
m.tt0 = Var(m.N)


m.h = Var(m.N, bounds =(0.1,2.0))

a = [[0.19681547722366, 0.39442431473909, 0.37640306270047],
     [-0.06553542585020, 0.29207341166523, 0.51248582618842],
     [0.02377097434822, -0.04154875212600, 0.11111111111111]]
print('Done')

Done


Here we define the parameters of the model
g=gravity
pi=pi

all the masses and inertia values (converting from symbol to numerical value)

In [11]:
# PARAMETERS-----------------------------------------------------------------------------------------------------------------
m.g = Param(initialize = 9.81)
pi=sym.pi

m.mb = Param(initialize = mass_b)
m.ml = Param(initialize = mass_l)
m.lb = Param(initialize = len_b)

m.Inb= Param(initialize = m.mb.value*m.lb.value**2/12)   
m.Inl= Param(initialize = m.ml.value*m.lb.value**2/12)  ##############################m.ml.value*m.q[m.n,m.cN,'r']**2/12

mbody = mass_b+mass_l
BW = mbody*m.g.value

print('Done')

Done


#### Collocation constraints
Here we implement the collocation constraints (see image above as to how q0 and q are linked)

In [12]:
def ContConstraint_p (m, n, dof):
    if n > 1:
        return m.q0[n,dof] == m.q[n-1,3,dof]
    else:
        return Constraint.Skip
m.ContConstraint_p = Constraint(m.N, m.DOF, rule = ContConstraint_p)
def ContConstraint_v (m, n, dof):
    if n > 1:
        return m.dq0[n,dof] == m.dq[n-1,3,dof]
    else:
        return Constraint.Skip
m.ContConstraint_v = Constraint(m.N, m.DOF, rule = ContConstraint_v)
def ContConstraint_tt (m, n):
    if n > 1:
        return m.tt0[n] == m.tt[n-1,3]
    else:
        return Constraint.Skip
m.ContConstraint_tt = Constraint(m.N, rule = ContConstraint_tt)
print('Done')

Done


#### Integration: 3 point collocation, Runge Kutta integration 
Here we implement the integration

In [13]:
def integrate_p(m, n, c, dof): # for positions
    if n > 0:
        return m.q[n,c,dof] == m.q0[n, dof] + hm*m.h[n]*(a[0][c-1]*m.dq[n,1,dof] \
                                                         +a[1][c-1]*m.dq[n,2,dof] \
                                                         +a[2][c-1]*m.dq[n,3,dof])
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.cN, m.DOF, rule = integrate_p)
def integrate_v(m,n,c,dof): # for velocities
    if n > 0:
        return m.dq[n,c,dof] == m.dq0[n, dof] + hm*m.h[n]*(a[0][c-1]*m.ddq[n,1,dof] \
                                                           +a[1][c-1]*m.ddq[n,2,dof] \
                                                           +a[2][c-1]*m.ddq[n,3,dof])
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.cN, m.DOF, rule = integrate_v)
def integrate_tt(m,n,c): # for time 
    if n > 0:
        return m.tt[n,c] == m.tt0[n] + hm*m.h[n]*(a[0][c-1]+a[1][c-1]+a[2][c-1])
    else:
        return Constraint.Skip 
m.integrate_tt = Constraint(m.N, m.cN, rule = integrate_tt)
print('Done')

Done


#### GRF stuff

Here we set up the + and - sign of the GRF. See: A direct method for trajectory optimization of rigid bodies through contact
https://journals.sagepub.com/doi/abs/10.1177/0278364913506757
OR
https://groups.csail.mit.edu/robotics-center/public_papers/Posa13.pdf
 
Look at equations 8 to 16. Eq 8 and 9 force positive variables. Eq 10, 11 and 12 are technically not needed. 
 
Eq 13 is handled as follows:
m.a_contact_1[n,c] == lamb_footz(*var_list)
m.b_contact_1[n,c] == m.GRF[n,c,'Z','ps']
a_contact_1_pr = m.a_contact_1[n,1] + m.a_contact_1[n,2] + m.a_contact_1[n,3] 
b_contact_1_pr = m.b_contact_1[n,1] + m.b_contact_1[n,2] + m.b_contact_1[n,3]  
return a_contact_1_pr*b_contact_1_pr <= m.eps
 
Eq 14 is handled as follows:
m.a_friction_1[n,c] == m.mu*m.GRF[n,c,'Z','ps'] - (m.GRF[n,c,'X','ps'] + m.GRF[n,c,'X','ng'])
m.b_friction_1[n,c] == m.gamma_1[n,c]
a_friction_1_pr = m.a_friction_1[n,1] + m.a_friction_1[n,2] + m.a_friction_1[n,3] 
b_friction_1_pr = m.b_friction_1[n,1] + m.b_friction_1[n,2] + m.b_friction_1[n,3]  
return a_friction_1_pr*b_friction_1_pr <= m.eps
 
Eq 15 is handled as follows:
m.a_slip_p_1[n,c] ==m.gamma_1[n,c]+lamb_footdx(*var_list)
m.b_slip_p_1[n,c] == m.GRF[n,c,'X','ps']
a_slip_p_1_pr = m.a_slip_p_1[n,1] + m.a_slip_p_1[n,2] + m.a_slip_p_1[n,3] 
b_slip_p_1_pr = m.b_slip_p_1[n,1] + m.b_slip_p_1[n,2] + m.b_slip_p_1[n,3]  
return a_slip_p_1_pr*b_slip_p_1_pr <= m.eps
 
Eq 16 is handled as follows: 
m.a_slip_n_1[n,c] ==m.gamma_1[n,c]-lamb_footdx(*var_list)
m.b_slip_n_1[n,c] == m.GRF[n,c,'X','ng']
a_slip_n_1_pr = m.a_slip_n_1[n,1] + m.a_slip_n_1[n,2] + m.a_slip_n_1[n,3] 
b_slip_n_1_pr = m.b_slip_n_1[n,1] + m.b_slip_n_1[n,2] + m.b_slip_n_1[n,3]  
return a_slip_n_1_pr*b_slip_n_1_pr <= m.eps



In [14]:
signs = ['ps', 'ng']
m.sgn = Set(initialize = ['ps', 'ng'])
WDOFs = ['X', 'Z','THETA1']
m.WDOF = Set(initialize=WDOFs) #change the coordinates
m.GRF = Var(m.N, m.cN, m.WDOF, m.sgn, bounds = (0.0,None)) # should bound it to 10*m_robot*g
m.eps = Param(initialize=1000.0, mutable=True)
m.mu = Param(initialize =1.00)
print('Done')

Done


define all the slack variables

In [15]:
# SLACK VARIABLES
m.a_contact_1  = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_contact_1  = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_friction_1 = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_friction_1 = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_slip_p_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_slip_p_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.a_slip_n_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.b_slip_n_1   = Var(m.N, m.cN, bounds = (0.0,None)) 
m.gamma_1      = Var(m.N, m.cN, bounds = (0.0,None)) 
print('Done')

Done


Handle the contact event

In [16]:
# CONTACT
def def_a_contact_1(m,n,c):   
        var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            0,0,0,0]
        return m.a_contact_1[n,c] == lamb_footz(*var_list)
m.def_a_contact_1 = Constraint(m.N, m.cN, rule = def_a_contact_1)
def def_b_contact_1(m,n,c):   
    return m.b_contact_1[n,c] == m.GRF[n,c,'Z','ps']
m.def_b_contact_1 = Constraint(m.N, m.cN, rule = def_b_contact_1)
print('Done')

Done


Handle the friction cone

In [17]:
# FRICTION CONE
def def_a_friction_1(m,n,c):   
    return m.a_friction_1[n,c] == m.mu*m.GRF[n,c,'Z','ps'] - (m.GRF[n,c,'X','ps'] + m.GRF[n,c,'X','ng'])
m.def_a_friction_1 = Constraint(m.N, m.cN, rule = def_a_friction_1)
def def_b_friction_1(m,n,c):   
    return m.b_friction_1[n,c] == m.gamma_1[n,c]
m.def_b_friction_1 = Constraint(m.N, m.cN, rule = def_b_friction_1)
print('Done')

Done


Handle the foot slipping (make sure it slips in the correct direction)

In [18]:
#SLIP POS DIR
def def_a_slip_p_1(m,n,c):   
    var_list = [m.g,m.mb,m.ml,
        m.lb,
        m.Inb,m.Inl,
        m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
        m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
        m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
        0,0,0,0]
    return m.a_slip_p_1[n,c] ==m.gamma_1[n,c]+lamb_footdx(*var_list)
m.def_a_slip_p_1 = Constraint(m.N, m.cN, rule = def_a_slip_p_1)
def def_b_slip_p_1(m,n,c):   
    return m.b_slip_p_1[n,c] == m.GRF[n,c,'X','ps']
m.def_b_slip_p_1 = Constraint(m.N, m.cN, rule = def_b_slip_p_1)

# SLIP NEG DIR
def def_a_slip_n_1(m,n,c):   
    var_list = [m.g,m.mb,m.ml,
        m.lb,
        m.Inb,m.Inl,
        m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
        m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
        m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
        0,0,0,0]
    return m.a_slip_n_1[n,c] ==m.gamma_1[n,c]-lamb_footdx(*var_list)
m.def_a_slip_n_1 = Constraint(m.N, m.cN, rule = def_a_slip_n_1)
def def_b_slip_n_1(m,n,c):   
    return m.b_slip_n_1[n,c] == m.GRF[n,c,'X','ng']
m.def_b_slip_n_1 = Constraint(m.N, m.cN, rule = def_b_slip_n_1)

print('Done')

Done


Sum the complimentarity parts across the collocation points and evaluate at the node points

In [19]:
def def_contact_1_pr(m,n):
    a_contact_1_pr = m.a_contact_1[n,1] + m.a_contact_1[n,2] + m.a_contact_1[n,3] 
    b_contact_1_pr = m.b_contact_1[n,1] + m.b_contact_1[n,2] + m.b_contact_1[n,3]  
    return a_contact_1_pr*b_contact_1_pr <= m.eps
m.def_contact_1_pr = Constraint(m.N, rule = def_contact_1_pr)

def def_friction_1_pr(m,n):
    a_friction_1_pr = m.a_friction_1[n,1] + m.a_friction_1[n,2] + m.a_friction_1[n,3] 
    b_friction_1_pr = m.b_friction_1[n,1] + m.b_friction_1[n,2] + m.b_friction_1[n,3]  
    return a_friction_1_pr*b_friction_1_pr <= m.eps
m.def_friction_1_pr = Constraint(m.N, rule = def_friction_1_pr)

def def_slip_p_1_pr(m,n):
    a_slip_p_1_pr = m.a_slip_p_1[n,1] + m.a_slip_p_1[n,2] + m.a_slip_p_1[n,3] 
    b_slip_p_1_pr = m.b_slip_p_1[n,1] + m.b_slip_p_1[n,2] + m.b_slip_p_1[n,3]  
    return a_slip_p_1_pr*b_slip_p_1_pr <= m.eps
m.def_slip_p_1_pr = Constraint(m.N, rule = def_slip_p_1_pr)

def def_slip_n_1_pr(m,n):
    a_slip_n_1_pr = m.a_slip_n_1[n,1] + m.a_slip_n_1[n,2] + m.a_slip_n_1[n,3] 
    b_slip_n_1_pr = m.b_slip_n_1[n,1] + m.b_slip_n_1[n,2] + m.b_slip_n_1[n,3]  
    return a_slip_n_1_pr*b_slip_n_1_pr <= m.eps
m.def_slip_n_1_pr = Constraint(m.N, rule = def_slip_n_1_pr)
print('Done')

Done


#### Motor model

Here we implement a motor model. In this example we implement it through bounds (see bounds section) and a liner motor model. See: Do Intermediate Gaits Matter When Rapidly Accelerating? https://static1.squarespace.com/static/542ddec8e4b0158794bd1036/t/5e3e0275c0b9914844da5cc0/1581122167965/Fisher_etal_2019.pdf eq(6)

In [20]:
m.tau_a = Var(m.N, bounds = (-m.t_max,m.t_max)) # actuator torque at hip
m.F_a = Var(m.N, bounds = (-m.F_max,m.F_max)) # actuator prismatic force

def def_hip_motor_model_p(m,n):
    return  m.tau_a[n] <= m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l']-m.dq0[n,'theta_b'])
m.def_hip_motor_model_p = Constraint(m.N, rule = def_hip_motor_model_p)
def def_hip_motor_model_n(m,n):
    return  m.tau_a[n] >= -m.t_max-m.t_max/m.w_max*(m.dq0[n,'theta_l']-m.dq0[n,'theta_b'])
m.def_hip_motor_model_n = Constraint(m.N, rule = def_hip_motor_model_n)

print('Done')

Done


#### EoM constraints
Make sure the EoM are satisfied

In [21]:
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

S=1.0 #BW/m.mTot    #Scaling factor, I avoid scaling the problem, others seem to think it works....

def EOM_x(m,n,c):
    F_in = S*(m.F_a[n])
    tau_in = S*(m.tau_a[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            F_in,tau_in,Gx_in,Gz_in]
    return lambEOM_x(*var_list) == 0
m.EOM_x = Constraint(m.N, m.cN, rule = EOM_x)

def EOM_y(m,n,c):
    F_in = S*(m.F_a[n])
    tau_in = S*(m.tau_a[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            F_in,tau_in,Gx_in,Gz_in]
    return lambEOM_y(*var_list) == 0
m.EOM_y = Constraint(m.N, m.cN, rule = EOM_y)

def EOM_thb(m,n,c):
    F_in = S*(m.F_a[n])
    tau_in = S*(m.tau_a[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            F_in,tau_in,Gx_in,Gz_in]
    return lambEOM_thb(*var_list) == 0
m.EOM_thb = Constraint(m.N, m.cN, rule = EOM_thb)

def EOM_thl(m,n,c):
    F_in = S*(m.F_a[n])
    tau_in = S*(m.tau_a[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            F_in,tau_in,Gx_in,Gz_in]
    return lambEOM_thl(*var_list) == 0
m.EOM_thl = Constraint(m.N, m.cN, rule = EOM_thl)

def EOM_r(m,n,c):
    F_in = S*(m.F_a[n])
    tau_in = S*(m.tau_a[n])
    Gx_in = S*(m.GRF[n,c,'X','ps']-m.GRF[n,c,'X','ng'])
    Gz_in = S*(m.GRF[n,c,'Z','ps'])
    
    var_list = [m.g,m.mb,m.ml,
            m.lb,
            m.Inb,m.Inl,
            m.q[n,c,'x'],m.q[n,c,'z'],m.q[n,c,'theta_b'],m.q[n,c,'theta_l'],m.q[n,c,'r'],
            m.dq[n,c,'x'],m.dq[n,c,'z'],m.dq[n,c,'theta_b'],m.dq[n,c,'theta_l'],m.dq[n,c,'r'],
            m.ddq[n,c,'x'],m.ddq[n,c,'z'],m.ddq[n,c,'theta_b'],m.ddq[n,c,'theta_l'],m.ddq[n,c,'r'],
            F_in,tau_in,Gx_in,Gz_in]
    return lambEOM_r(*var_list) == 0
m.EOM_r = Constraint(m.N, m.cN, rule = EOM_r)
print('Done')

Done


#### Bounds
Bound all variables here. Bound to either enforce something (eg leg length) or bound sufficiently high to reduce the search space but not effect the solution. For example if robot runs from 0m to 5m, bound the x variable to -1m to 6m. This does not effect the solution space, but drastically reduces the search space (x was -inf to inf).

In [22]:
def setxBounds(m,n):
    return (-1.0, m.q0[n,'x'], m.distance+1)
m.setxBounds = Constraint(m.N, rule=setxBounds)
def setzBounds(m,n):
    return (0.0, m.q0[n,'z'], 1.0)
m.setzBounds = Constraint(m.N, rule=setzBounds)
def setthbBounds(m,n):
    return (-3.14/2, m.q0[n,'theta_b'], 3.14/2)
m.setthbBounds = Constraint(m.N, rule=setthbBounds)
def setthlBounds(m,n):
    return (0, m.q0[n,'theta_l'], 3.14)
m.setthlBounds = Constraint(m.N, rule=setthlBounds)
def setrBounds(m,n):
    return (m.l_min, m.q0[n,'r'], m.l_max)
m.setrBounds = Constraint(m.N, rule=setrBounds)

def setdxBounds(m,n):
    return (-1, m.dq0[n,'x'],10)
m.setdxBounds = Constraint(m.N, rule=setdxBounds)
def setdzBounds(m,n):
    return (-10, m.dq0[n,'z'],10)
m.setdzBounds = Constraint(m.N, rule=setdzBounds)
def setdthbBounds(m,n):
    return (-20, m.dq0[n,'theta_b'], 20)
m.setdthbBounds = Constraint(m.N, rule=setdthbBounds)
def setdthlBounds(m,n):
    return (-m.w_max, m.dq0[n,'theta_l'], m.w_max)
m.setdthlBounds = Constraint(m.N, rule=setdthlBounds)
def setdrBounds(m,n):
    return (-m.dl_max, m.dq0[n,'r'],m.dl_max)
m.setdrBounds = Constraint(m.N, rule=setdrBounds)

print('Done')

Done


#### Seed generation
In order to search the solution space, a random seed point (start point) is generated. This means that occasionally you will not get a solution, and if you do, it may be a bad solution. In that case, generate a bunch of seeds or start the optimizer with a good seed point

In [23]:
R = 1.0
D = 5.0

Domain = np.linspace(0,D,N)
Range = np.linspace(0,R,N)
for n in range(1,N+1):
    for c in range (1, cN+1): 
        #Randomizing Q Matrix
        m.q[n,c,'x'].value = Domain[n-1] + np.random.uniform(-0.25,0.25)    
        m.q[n,c,'z'].value  = Range[n-1] + np.random.uniform(-0.25,0.25)
        m.q[n,c,'theta_b'].value  = np.random.uniform(0.0,np.pi/2)
        m.q[n,c,'theta_l'].value  = np.random.uniform(0.0,np.pi/2)
        m.q[n,c,'r'].value  = np.random.uniform(0.0,0.5)
        
        #Setting everything else to 0.01
        for dof in DOFs:
            m.dq[n,c , dof].value = 0.01
            m.ddq[n,c , dof].value = 0.01


        m.GRF[n,c,'Z','ps'].value = 0.01
        m.GRF[n,c,'X','ps'].value = 0.01 
        m.GRF[n,c,'X','ng'].value = 0.01 
        
print('Done')

Done


#### End conditions
In order to enforce a steady-state step, a number of start and end conditions are implemented
generally start conditions are enforced through bounds and end conditions are enforced through constraints.

In [24]:
# constraints for steady-state 
m.q[1,1,'x'].value = 0.0
m.q[1,1,'x'].fixed = True
m.dq[1,1,'z'].value = 0.0
m.dq[1,1,'z'].fixed = True

def finalX(m,n):
    if (n==N) :
        return m.q0[n,'x'] >= m.distance
    else:
        return Constraint.Skip
m.finalX = Constraint(m.N, rule = finalX)

def finalZ(m,n):
    if (n==N) :
        return m.q0[n,'z'] == m.q0[1,'z']
    else:
        return Constraint.Skip
m.finalZ = Constraint(m.N, rule = finalZ)


def finalthb(m,n):
    if (n==N) :
        return m.q0[n,'theta_b'] == m.q0[1,'theta_b']
    else:
        return Constraint.Skip
m.finalthb = Constraint(m.N, rule = finalthb)


def finalthl(m,n):
    if (n==N) :
        return m.q0[n,'theta_l'] == m.q0[1,'theta_l']
    else:
        return Constraint.Skip
m.finalthl = Constraint(m.N, rule = finalthl)

def finalr(m,n):
    if (n==N) :
        return m.q0[n,'r'] == m.q0[1,'r']
    else:
        return Constraint.Skip
m.finalr = Constraint(m.N, rule = finalr)


def finaldx(m,n):
    if (n==N) :
        return m.dq0[n,'x'] == m.dq0[1,'x']
    else:
        return Constraint.Skip
m.finaldx = Constraint(m.N, rule = finaldx)

def finaldz(m,n):
    if (n==N) :
        return m.dq0[n,'z'] == m.dq0[1,'z']
    else:
        return Constraint.Skip
m.finaldz = Constraint(m.N, rule = finaldz)

def finaldthb(m,n):
    if (n==N) :
        return m.dq0[n,'theta_b'] == m.dq0[1,'theta_b']
    else:
        return Constraint.Skip
m.finaldthb = Constraint(m.N, rule = finaldthb)


def finaldthl(m,n):
    if (n==N) :
        return m.dq0[n,'theta_l'] == m.dq0[1,'theta_l']
    else:
        return Constraint.Skip
m.finaldthl = Constraint(m.N, rule = finaldthl)

def finaldr(m,n):
    if (n==N) :
        return m.dq0[n,'r'] == m.dq0[1,'r']
    else:
        return Constraint.Skip
m.finaldr = Constraint(m.N, rule = finaldr)
print('Done')

Done


#### Run the solver

First define the cost. We set the cost to 1, solve the seed, then set the cost to the correct value and re-solve in a loop. 
after each solve, reduce the epsilon value and solve again until within good tolerance

In [ ]:
opt = SolverFactory('ipopt') # standard issue, garden variety ipopt
opt.options["print_level"] = 5 # was 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 100000 # maximum number of iterations
opt.options["max_cpu_time"] = 120000 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.


def CostFun(m):
    return 1 
m.Cost = Objective(rule = CostFun)

#solve once with obj=1
results = opt.solve(m, tee=True)
def CostFun(m):
    return sum(((m.tau_a[n]**2+m.F_a[n]**2)) for n in range(1,N+1)) #Minimum total actuator force and torque
m.Cost = Objective(rule = CostFun)

#loop solve with proper objective
for i in range(0,4):
    print(m.eps.value)
    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        #If the previous solution is optmial, continue solving
        results = opt.solve(m, tee=True)
        m.eps.value = m.eps.value/10
    elif (results.solver.termination_condition == TerminationCondition.infeasible):
        # Break loop when model in infeasible
        break

Ipopt 3.13.2: print_level=5
max_iter=100000
max_cpu_time=120000
Tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    14831
Number of nonzeros in inequality constraint Jacobian.:     1401
Number of nonzeros in Lagrangian Hessian.............:     4619

Total number of variables............................:     3428
                     variables with only lower bounds:     1260
                variables with lower and upper bounds:      105
         

### Animate the solution








THIS DOES NOT PRINT WRT TIME AND DOES NOT LOOK AT THE COLLOCATION POINTS ONLY THE NODE POINTS 




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

print(results.solver.status) 
print(results.solver.termination_condition) 

fig1, ax1 = plt.subplots(1,1) #create axes
#ax1.set_aspect('equal')

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-0.5,1])
    ax.set_ylim([0,1])
    
    #plot body
    body_xb = m.q0[i,'x'].value - 0.5*m.lb*cos(m.q0[i,'theta_b'].value)
    body_yb = m.q0[i,'z'].value - 0.5*m.lb*sin(m.q0[i,'theta_b'].value)
    body_xf = m.q0[i,'x'].value + 0.5*m.lb*cos(m.q0[i,'theta_b'].value)
    body_yf = m.q0[i,'z'].value + 0.5*m.lb*sin(m.q0[i,'theta_b'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
    
    #plot leg 
    leg_xt = m.q0[i,'x'].value 
    leg_yt = m.q0[i,'z'].value 
    leg_xb = m.q0[i,'x'].value + m.q0[i,'r'].value*cos(m.q0[i,'theta_l'].value)
    leg_yb = m.q0[i,'z'].value - m.q0[i,'r'].value*sin(m.q0[i,'theta_l'].value)
    ax.plot([leg_xt,leg_xb],[leg_yt,leg_yb],color='xkcd:black')
    
update = lambda i: plot_robot(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=False)

HTML(animate.to_jshtml())

In [ ]:
'''from numpy import asarray
from numpy import save

data = asarray([[N for n in range (1, N+1)],
                [m.lb.value for n in range (1, N+1)],
                [m.lb.value for n in range(1, N+1)],
                [m.tau_a[n].value for n in range (1, N+1)],
                [m.F_a[n].value for n in range (1, N+1)],
                [m.q0[n,'theta_l'].value for n in range (1, N+1)],
                [m.q0[n,'r'].value for n in range (1, N+1)],
                [m.q0[n,'x'].value for n in range (1, N+1)],
                [m.q0[n,'z'].value for n in range (1, N+1)]])
save('data.npy', data)'''

